In [1]:
"""
ATFM
@author: xxx
"""
# Make sure this libraries are installed in your virtual environment
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
from pyomo.environ import *
from coopr.pyomo import *

In [2]:
# Elements of the problem

#Dimensions
n_airports = 3
n_sectors = 4
n_flights = 3
time_intervals = 40

#Sets
T = list(range(time_intervals+1))
F = ['f1', 'f2', 'f3']
K = ['A', 'B', 'C']
S = ['I', 'II', 'III', 'IV']
C = [F[1], F[2]]

# Departure capacity of airport K at time t
D_k = {}
def D_k_ini(D_k, time_intervals):
    for i in range(1, time_intervals+1):
        D_k["A",i] = 1
        D_k["B",i] = 1
        D_k["C",i] = 1
    return D_k

# Arrival capacity of airport K at time t
A_k = {}
def A_k_ini(A_k, time_intervals):
    for i in range(1, time_intervals+1):
        A_k["A",i] = 1
        A_k["B",i] = 1
        A_k["C",i] = 1
    return A_k

# Capacity of sector j at time t
S_j = {}
def S_j_ini(S_j, time_intervals):
    for i in range(1, time_intervals+1):
        S_j["I",i]   = 4
        S_j["II",i]  = 4
        S_j["III",i] = 4
        S_j["IV",i]  = 4
    return S_j


# Scheduled departure times
d_f = {F[0]: 1, F[1]: 1, F[2]: 17}

# Scheduled arrivals times
a_f = {F[0]: 15, F[1]: 13, F[2]: 23}

# Turnaround times
s_f = {F[0]: 5, F[1]: 5, F[2]: 5}

# Origin and Destination Airport
origin_f={}
origin_f[F[0]]= K[0]
origin_f[F[1]]= K[0]
origin_f[F[2]]= K[1]

destination_f={}
destination_f[F[0]]= K[2]
destination_f[F[1]]= K[1]
destination_f[F[2]]= K[2]

# Nodes for flight f
def N_f_ini(N_f):
    N_f["f1"] = ("A","q(A)","a","b","C")
    N_f["f2"] = ("A","q(A)","e","B")
    N_f["f3"] = ("B","q(B)","f","C")
    return N_f
N_f = {}

# Arcs for flight f
def arcs_ini(arcs):
    arcs["f1"] = (  "(A,q(A))","(q(A),a)","(a,b)","(b,C)"  )
    arcs["f2"] = (  "(A,q(A))","(q(A),e)","(e,B)"  )
    arcs["f3"] = (  "(B,q(B))","(q(B),f)","(f,C)"  )
    return arcs
arcs = {}

# Number of time units that flight f must spend in arc m,n
def l_fnm_ini(l_fnm):
    l_fnm["f1", "(A,q(A))"] = 0
    l_fnm["f1", "(q(A),a)"] = 5
    l_fnm["f1", "(a,b)"]    = 8
    l_fnm["f1", "(b,C)"]    = 1
    l_fnm["f2", "(A,q(A))"] = 0
    l_fnm["f2", "(q(A),e)"] = 5
    l_fnm["f2", "(e,B)"]    = 7
    l_fnm["f3", "(B,q(B))"] = 0
    l_fnm["f3", "(q(B),f)"] = 3
    l_fnm["f3", "(f,C)"]    = 3
    return l_fnm
l_fnm = {}

# Scheduled time of arrival of flight f to arrive at node n (not considering possible delays)
def r_fn_ini(r_fn,model):
    for k in model.F:
        for i1,val1 in enumerate(model.N_f[k]):
            r_fn[k, val1] = model.d_f[k]
            for i2, val2 in enumerate(model.arcs[k]):
                if i2<i1:
                    r_fn[k, val1] = r_fn[k, val1]+model.l_fnm[k,val2]
    return r_fn
r_fn = {}

# maximum ground holding time units delay per flight f
def G_ini(G):
    G["f1"] = 2
    G["f2"] = 3
    G["f3"] = 3
    return G
G = {}

# maximum airborne unit delays for flight f
def A_ini(A):
    A["f1"] = 5
    A["f2"] = 4
    A["f3"] = 4
    return A
A = {}

# Smallest time interval for flight f to arrive at node n
def T_fn_ini(T_fn,model):
    for k in model.F:
        for i,val in enumerate(model.N_f[k]):
            if i<2:
                T_fn[k,val] = np.linspace(model.r_fn[k,val], min(model.r_fn[k,val]+model.G[k],time_intervals) , min(model.r_fn[k,val]+model.G[k],time_intervals)-model.r_fn[k,val]+1)
            if i>=2:
                T_fn[k, val] = np.linspace(model.r_fn[k,val], min(model.r_fn[k,val]+model.G[k]+model.A[k],time_intervals) , min(model.r_fn[k,val]+model.G[k]+model.A[k],time_intervals)-model.r_fn[k,val]+1)
    return T_fn
T_fn = {}

# Unit cost for air delay
def ca_ini(ca):
    ca["f1"] = 3
    ca["f2"] = 2.5
    ca["f3"] = 2.5
    return ca
ca = {}

# Unit cost for ground delay
def cg_ini(cg):
    cg["f1"] = 1.5
    cg["f2"] = 1.5
    cg["f3"] = 1.5
    return cg
cg = {}

# Nodes and arcs belonging to Sector j that are overflown by Flight k
# [Note that N_S denotes what in the statement is N_{jf}]
def N_S_ini(N_S):
    N_S["f1", "I"]   = "A", "q(A)", "a"
    N_S["f1", "II"]  = "a", "b"
    N_S["f1", "III"] = ""
    N_S["f1", "IV"]  = "b", "C"
    N_S["f2", "I"]   = "A", "q(A)", "e"
    N_S["f2", "II"]  = ""
    N_S["f2", "III"] = "e", "B"
    N_S["f2", "IV"]  = ""
    N_S["f3", "I"]   = ""
    N_S["f3", "II"]  = ""
    N_S["f3", "III"] = "B", "q(B)", "f"
    N_S["f3", "IV"]  = "f", "C"
    return N_S
N_S = {}

def arc_S_ini(arc_S):
    arc_S["f1", "I"]    = "(A,q(A))", "(q(A),a)"
    arc_S["f1", "II"]   = "(a,b)"
    arc_S["f1", "III"]  = {}
    arc_S["f1", "IV"]   = "(b,C)"
    arc_S["f2", "I"]    = "(A,q(A))", "(q(A),e)"
    arc_S["f2", "II"]   = ""
    arc_S["f2", "III"]  = "(e,B)"
    arc_S["f2", "IV"]   = ""
    arc_S["f3", "I"]    = ""
    arc_S["f3", "II"]   = ""
    arc_S["f3", "III"]  = "(B,q(B))", "(q(B),f)"
    arc_S["f3", "IV"]   = "(f,C)"
    return arc_S
arc_S = {}

print('origin airport:', origin_f)
print('dstination airport:', destination_f)
print('Times', T)
print('Nodes in Sector S', N_S)
print('Arcs in Sector S', arc_S)
print('Set of Times to overfly node n for flight f', T_fn)
print('Nf', N_f)

origin airport: {'f1': 'A', 'f2': 'A', 'f3': 'B'}
dstination airport: {'f1': 'C', 'f2': 'B', 'f3': 'C'}
Times [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]
Nodes in Sector S {}
Arcs in Sector S {}
Set of Times to overfly node n for flight f {}
Nf {}


In [3]:
############### Create a concrete model ###############
model = ConcreteModel('ATFM')

In [ ]:
############### Initialize Sets and Parameters ###############
############### FEW examples are given below (complete until you get the print that you will find in the cell below) ###############

model.T = Set(initialize=T, ordered=True)
model.N_f = Set(N_f.keys(), initialize=N_f_ini(N_f), ordered=True)
model.T_fn = Set(T_fn.keys(), initialize=T_fn_ini(T_fn,model))

print('T_fn:', T_fn)
print('N_f:', N_f)

# You should get this print
T_fn: {('f1', 'A'): array([1., 2., 3.]), ('f1', 'q(A)'): array([1., 2., 3.]), ('f1', 'a'): array([ 6.,  7.,  8.,  9., 10., 11., 12., 13.]), ('f1', 'b'): array([14., 15., 16., 17., 18., 19., 20., 21.]), ('f1', 'C'): array([15., 16., 17., 18., 19., 20., 21., 22.]), ('f2', 'A'): array([1., 2., 3., 4.]), ('f2', 'q(A)'): array([1., 2., 3., 4.]), ('f2', 'e'): array([ 6.,  7.,  8.,  9., 10., 11., 12., 13.]), ('f2', 'B'): array([13., 14., 15., 16., 17., 18., 19., 20.]), ('f3', 'B'): array([17., 18., 19., 20.]), ('f3', 'q(B)'): array([17., 18., 19., 20.]), ('f3', 'f'): array([20., 21., 22., 23., 24., 25., 26., 27.]), ('f3', 'C'): array([23., 24., 25., 26., 27., 28., 29., 30.])}

N_f: {'f1': ('A', 'q(A)', 'a', 'b', 'C'), 'f2': ('A', 'q(A)', 'e', 'B'), 'f3': ('B', 'q(B)', 'f', 'C')}

In [ ]:
############### Define Variables ###############

var = {}
var["f1","A"] = ""
# complete the definition of the variables until you get the print that you will find in the cell below

model.w = Var(var, model.T, within=Binary)

In [76]:
### Define Objective Function ###


In [77]:
################### Define Departure Capacity constraint ####################

def departure_capacity_constraint(model,k,t):
    if t ==0:
        return Constraint.Skip #This is to skip the constratin at t=0.
    
    # Check if any flight has origin 'k'
    has_flights_from_k = any(model.origin_f[f] == k for f in model.F)
    
    #Complete the rest of the instructions for the constraint.



In [78]:
################### Define Arrival Capacity constraint ####################


In [ ]:
################### Define Sector Capacity constraint ####################
# For debugging purposes
for index, value in enumerate(model.N_S["f1", "I"]):
    print(f"Index: {index}, Value: {value}")



# You should get this print
Index: 0, Value: A
Index: 1, Value: q(A)
Index: 2, Value: a

In [ ]:
######## Connectivity constratins 1 ########
# Create a set that combines the indices you need for the constraint
model.T_fn_combined = Set(dimen=3, initialize=lambda model: ((f, n, t) for f in model.F for n in model.N_f[f] for t in model.T_fn[f,n]))
display(model.T_fn_combined) 

def flight_structure_constraints_1_rule(model, f, j, t):
    #complete the constraint.
    # It may be interesting to access to the index (n) of a given element j) 
    #n = N_f[f].index(j)
    # It may be also interesting to access to the element (m) of a fiven index (n + 1)
    #m = N_f[f][n+1]
    
model.flight_structure_constraints_1 = Constraint(model.T_fn_combined, rule=flight_structure_constraints_1_rule)


You should get this print:

T_fn_combined : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     3 :    Any :   78 : {('f1', 'A', 1.0), ('f1', 'A', 2.0), ('f1', 'A', 3.0), ('f1', 'q(A)', 1.0), ('f1', 'q(A)', 2.0), ('f1', 'q(A)', 3.0), ('f1', 'a', 6.0), ('f1', 'a', 7.0), ('f1', 'a', 8.0), ('f1', 'a', 9.0), ('f1', 'a', 10.0), ('f1', 'a', 11.0), ('f1', 'a', 12.0), ('f1', 'a', 13.0), ('f1', 'b', 14.0), ('f1', 'b', 15.0), ('f1', 'b', 16.0), ('f1', 'b', 17.0), ('f1', 'b', 18.0), ('f1', 'b', 19.0), ('f1', 'b', 20.0), ('f1', 'b', 21.0), ('f1', 'C', 15.0), ('f1', 'C', 16.0), ('f1', 'C', 17.0), ('f1', 'C', 18.0), ('f1', 'C', 19.0), ('f1', 'C', 20.0), ('f1', 'C', 21.0), ('f1', 'C', 22.0), ('f2', 'A', 1.0), ('f2', 'A', 2.0), ('f2', 'A', 3.0), ('f2', 'A', 4.0), ('f2', 'q(A)', 1.0), ('f2', 'q(A)', 2.0), ('f2', 'q(A)', 3.0), ('f2', 'q(A)', 4.0), ('f2', 'e', 6.0), ('f2', 'e', 7.0), ('f2', 'e', 8.0), ('f2', 'e', 9.0), ('f2', 'e', 10.0), ('f2', 'e', 11.0), ('f2', 'e', 12.0), ('f2', 'e', 13.0), ('f2', 'B', 13.0), ('f2', 'B', 14.0), ('f2', 'B', 15.0), ('f2', 'B', 16.0), ('f2', 'B', 17.0), ('f2', 'B', 18.0), ('f2', 'B', 19.0), ('f2', 'B', 20.0), ('f3', 'B', 17.0), ('f3', 'B', 18.0), ('f3', 'B', 19.0), ('f3', 'B', 20.0), ('f3', 'q(B)', 17.0), ('f3', 'q(B)', 18.0), ('f3', 'q(B)', 19.0), ('f3', 'q(B)', 20.0), ('f3', 'f', 20.0), ('f3', 'f', 21.0), ('f3', 'f', 22.0), ('f3', 'f', 23.0), ('f3', 'f', 24.0), ('f3', 'f', 25.0), ('f3', 'f', 26.0), ('f3', 'f', 27.0), ('f3', 'C', 23.0), ('f3', 'C', 24.0), ('f3', 'C', 25.0), ('f3', 'C', 26.0), ('f3', 'C', 27.0), ('f3', 'C', 28.0), ('f3', 'C', 29.0), ('f3', 'C', 30.0)}
0 A q(A) (A,q(A))
0 A q(A) (A,q(A))
0 A q(A) (A,q(A))
1 q(A) a (q(A),a)
1 q(A) a (q(A),a)
1 q(A) a (q(A),a)
2 a b (a,b)
2 a b (a,b)
2 a b (a,b)
2 a b (a,b)
2 a b (a,b)
2 a b (a,b)
2 a b (a,b)
2 a b (a,b)
3 b C (b,C)
3 b C (b,C)
3 b C (b,C)
3 b C (b,C)
3 b C (b,C)
3 b C (b,C)
3 b C (b,C)
3 b C (b,C)
0 A q(A) (A,q(A))
0 A q(A) (A,q(A))
0 A q(A) (A,q(A))
0 A q(A) (A,q(A))
1 q(A) e (q(A),e)
1 q(A) e (q(A),e)
1 q(A) e (q(A),e)
1 q(A) e (q(A),e)
2 e B (e,B)
2 e B (e,B)
2 e B (e,B)
2 e B (e,B)
2 e B (e,B)
2 e B (e,B)
2 e B (e,B)
2 e B (e,B)
0 B q(B) (B,q(B))
0 B q(B) (B,q(B))
0 B q(B) (B,q(B))
0 B q(B) (B,q(B))
1 q(B) f (q(B),f)
1 q(B) f (q(B),f)
1 q(B) f (q(B),f)
1 q(B) f (q(B),f)
2 f C (f,C)
2 f C (f,C)
2 f C (f,C)
2 f C (f,C)
2 f C (f,C)
2 f C (f,C)
2 f C (f,C)
2 f C (f,C)

In [81]:
######## Connectivity constratins 2 ########



In [ ]:
######## Connectivity constratins 3 ########


In [ ]:
######## Connectivity constratins 4 ########
# It may be interesting to access to the first and last elements of a set 
#model.T_fn[f, j].first()
#model.T_fn[f, j].last()

# Note, please, that you need to do it not only for T_fn, yet for T. 


In [ ]:
######## Connectivity constratins 5 ########

# Create a set that combines the indices you need for the constraint
model.N_f_combined = Set(dimen=2, initialize=lambda model: ((f, n) for f in model.F for n in model.N_f[f]))
display(model.N_f_combined) 


In [ ]:
# Additional constraint to enforce W = 0 for all t < T_fn[f, j].first()

# Create a set that combines the indices you need for the constraint
model.N_f_combined = Set(dimen=2, initialize=lambda model: ((f, n) for f in model.F for n in model.N_f[f]))
display(model.N_f_combined) 

def connectivity_constraint8(model, f, j, t):
    if t <= model.T_fn[f, j].first() - 1:
        return model.w[f, j, t]==0
    else:
        return Constraint.Skip
model.connectivity_constraint8 = Constraint(model.N_f_combined, model.T, rule=connectivity_constraint8)
model.connectivity_constraint8.pprint()

In [ ]:
# Solver call
solver = SolverFactory('glpk')
#solver = SolverFactory('cplex_direct')
result = solver.solve(model)
model.display()
pyomo.environ.value(model.obj)

In [ ]:
# Prints: one can print the complete model or just some elements of it

model.pprint()
#model.obj.pprint()
#model.departure_capacity_constraint.pprint()
#model.arrival_capacity_constraint.pprint()
#model.sector_capacity_constraint.pprint()
#model.flight_structure_constraints_1.pprint()
#model.Connectivity2.pprint()
#model.Connectivity3_bis.pprint()
#model.flight_structure_constraints_4.pprint()
#model.connectivity_constraint4_bis.pprint()
#model.flight_structure_constraints_5.pprint()
model.w.pprint()

In [ ]:
#plots and Tables
